In [1]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [3]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [4]:
df = df.iloc[:300]

# Q1. Getting the embeddings model
##### Now, get the embeddings model multi-qa-mpnet-base-dot-v1 from the Sentence Transformer library

##### Note: this is not the same model as in HW3

##### from sentence_transformers import SentenceTransformer
##### embedding_model = SentenceTransformer(model_name)
##### Create the embeddings for the first LLM answer:

##### answer_llm = df.iloc[0].answer_llm
##### What's the first value of the resulting vector?

In [5]:
model_name = 'multi-qa-mpnet-base-dot-v1'

In [6]:
from sentence_transformers import SentenceTransformer

In [7]:
embedding_model = SentenceTransformer(model_name)

/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
ans1= embedding_model.encode(df.iloc[0].answer_llm)[0]

In [9]:
ans1

np.float32(-0.42244655)

# Q2. Computing the dot product
##### Now for each answer pair, let's create embeddings and compute dot product between them

##### We will put the results (scores) into the evaluations list

##### What's the 75% percentile of the score?

* 21.67
* 31.67
* 41.67
* 51.67

In [10]:
embedding_llm = []
for i in tqdm(df['answer_llm']):
    em = embedding_model.encode(i)
    embedding_llm.append(em)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:02<00:00,  4.83it/s]


In [11]:
embedding_orig = []
for i in tqdm(df['answer_orig']):
    em = embedding_model.encode(i)
    embedding_orig.append(em)
    

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:28<00:00,  3.39it/s]


In [13]:
df['answer_llm_v'] = embedding_llm

In [14]:
df['answer_orig_v'] = embedding_orig

In [15]:
df.head()

,answer_llm,answer_orig,document,question,course,answer_llm_v,answer_orig_v
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp,"[-0.42244655, -0.22485626, -0.3240584, -0.2847...","[-0.030214058, -0.3444381, -0.28076234, 0.0615..."
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp,"[-0.38068146, 0.047848288, -0.31510952, -0.210...","[-0.030214058, -0.3444381, -0.28076234, 0.0615..."
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,"[-0.05881373, -0.33736944, -0.36157572, 0.0217...","[-0.030214058, -0.3444381, -0.28076234, 0.0615..."
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,"[-0.22753648, -0.008134096, -0.21719913, -0.11...","[-0.030214058, -0.3444381, -0.28076234, 0.0615..."
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp,"[-0.06969386, -0.5005093, -0.1659844, 0.306661...","[-0.030214058, -0.3444381, -0.28076234, 0.0615..."


In [16]:
similarity = []
for i in tqdm(range(300)):
    sim = df['answer_llm_v'][i].dot(df['answer_orig_v'][i])
    similarity.append(sim)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 63534.02it/s]


In [17]:
df['similarity'] = similarity

In [18]:
df.head()

,answer_llm,answer_orig,document,question,course,answer_llm_v,answer_orig_v,similarity
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp,"[-0.42244655, -0.22485626, -0.3240584, -0.2847...","[-0.030214058, -0.3444381, -0.28076234, 0.0615...",17.515987
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp,"[-0.38068146, 0.047848288, -0.31510952, -0.210...","[-0.030214058, -0.3444381, -0.28076234, 0.0615...",13.418402
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,"[-0.05881373, -0.33736944, -0.36157572, 0.0217...","[-0.030214058, -0.3444381, -0.28076234, 0.0615...",25.313255
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,"[-0.22753648, -0.008134096, -0.21719913, -0.11...","[-0.030214058, -0.3444381, -0.28076234, 0.0615...",12.147415
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp,"[-0.06969386, -0.5005093, -0.1659844, 0.306661...","[-0.030214058, -0.3444381, -0.28076234, 0.0615...",18.747736


In [19]:
df['similarity'].describe()

count    300.000000
mean      27.495996
std        6.384742
min        4.547924
25%       24.307844
50%       28.336870
75%       31.674309
max       39.476013
Name: similarity, dtype: float64

# Q3. Computing the cosine
##### From Q2, we can see that the results are not within the [0, 1] range. It's because the vectors coming from this model are not normalized.

##### So we need to normalize them.

##### To do it, we

##### Compute the norm of a vector
##### Divide each element by this norm
##### So, for vector v, it'll be v / ||v||

##### In numpy, this is how you do it:

##### norm = np.sqrt((v * v).sum())
##### v_norm = v / norm
##### Let's put it into a function and then compute dot product between normalized vectors. This will give us cosine similarity

##### What's the 75% cosine in the scores?

* 0.63
* 0.73
* 0.83
* 0.93

In [22]:
def norm(vector):
    n = np.sqrt((vector*vector).sum())
    v_norm = vector/n
    return v_norm

In [26]:
df['answer_llm_v'][0].dot(df['answer_orig_v'][0])

np.float32(17.515987)

In [25]:
norm(df['answer_llm_v'][0]).dot(norm(df['answer_orig_v'][0]))

np.float32(0.5067539)

In [27]:
similarity_n = []
for i in tqdm(range(300)):
    sim2 = norm(df['answer_llm_v'][i]).dot(norm(df['answer_orig_v'][i]))
    similarity_n.append(sim2)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 33891.54it/s]


In [29]:
df['similarity_norm'] = similarity_n

In [30]:
df.head()

,answer_llm,answer_orig,document,question,course,answer_llm_v,answer_orig_v,similarity,similarity_norm
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp,"[-0.42244655, -0.22485626, -0.3240584, -0.2847...","[-0.030214058, -0.3444381, -0.28076234, 0.0615...",17.515987,0.506754
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp,"[-0.38068146, 0.047848288, -0.31510952, -0.210...","[-0.030214058, -0.3444381, -0.28076234, 0.0615...",13.418402,0.388549
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,"[-0.05881373, -0.33736944, -0.36157572, 0.0217...","[-0.030214058, -0.3444381, -0.28076234, 0.0615...",25.313255,0.718599
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,"[-0.22753648, -0.008134096, -0.21719913, -0.11...","[-0.030214058, -0.3444381, -0.28076234, 0.0615...",12.147415,0.337266
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp,"[-0.06969386, -0.5005093, -0.1659844, 0.306661...","[-0.030214058, -0.3444381, -0.28076234, 0.0615...",18.747736,0.521792


In [31]:
df['similarity_norm'].describe()

count    300.000000
mean       0.728393
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: similarity_norm, dtype: float64

# Q4. Rouge
##### Now we will explore an alternative metric - the ROUGE score.

##### This is a set of metrics that compares two answers based on the overlap of n-grams, word sequences, and word pairs.

##### It can give a more nuanced view of text similarity than just cosine similarity alone.

##### We don't need to implement it ourselves, there's a python package for it:

##### pip install rouge
##### (The latest version at the moment of writing is 1.0.1)

##### Let's compute the ROUGE score between the answers at the index 10 of our dataframe (doc_id=5170565b)

##### from rouge import Rouge
##### rouge_scorer = Rouge()

##### scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
##### There are three scores: rouge-1, rouge-2 and rouge-l, and precision, recall and F1 score for each.

##### rouge-1 - the overlap of unigrams,
##### rouge-2 - bigrams,
##### rouge-l - the longest common subsequence
##### What's the F score for rouge-1?

* 0.35
* 0.45
* 0.55
* 0.65

In [32]:
pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [33]:
from rouge import Rouge

In [36]:
rouge_scorer = Rouge()

In [40]:
scores = rouge_scorer.get_scores(df['answer_llm'][10], df['answer_orig'][10])[0]

In [41]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [43]:
scores['rouge-1']['f']

0.45454544954545456

# Q5. Average rouge score
##### Let's compute the average F-score between rouge-1, rouge-2 and rouge-l for the same record from Q4

* 0.35
* 0.45
* 0.55
* 0.65

In [47]:
(scores['rouge-1']['f']+ scores['rouge-2']['f']+ scores['rouge-l']['f'])/3

0.35490034990035496

# Q6. Average rouge score for all the data points
##### Now let's compute the F-score for all the records and create a dataframe from them.

##### What's the average F-score in rouge_2 across all the records?

* 0.10
* 0.20
* 0.30
* 0.40

In [48]:
rouge_2f = []
for i in tqdm(range(300)):
    scores2f = rouge_scorer.get_scores(df['answer_llm'][i], df['answer_orig'][i])[0]
    rouge_2f.append(scores2f['rouge-2']['f'])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 370.02it/s]


In [49]:
df['rouge_2f'] = rouge_2f

In [50]:
df.head()

,answer_llm,answer_orig,document,question,course,answer_llm_v,answer_orig_v,similarity,similarity_norm,rouge_2f
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp,"[-0.42244655, -0.22485626, -0.3240584, -0.2847...","[-0.030214058, -0.3444381, -0.28076234, 0.0615...",17.515987,0.506754,0.028169
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp,"[-0.38068146, 0.047848288, -0.31510952, -0.210...","[-0.030214058, -0.3444381, -0.28076234, 0.0615...",13.418402,0.388549,0.055556
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,"[-0.05881373, -0.33736944, -0.36157572, 0.0217...","[-0.030214058, -0.3444381, -0.28076234, 0.0615...",25.313255,0.718599,0.177778
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,"[-0.22753648, -0.008134096, -0.21719913, -0.11...","[-0.030214058, -0.3444381, -0.28076234, 0.0615...",12.147415,0.337266,0.047059
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp,"[-0.06969386, -0.5005093, -0.1659844, 0.306661...","[-0.030214058, -0.3444381, -0.28076234, 0.0615...",18.747736,0.521792,0.033898


In [51]:
df['rouge_2f'].describe()

count    300.000000
mean       0.206965
std        0.153550
min        0.000000
25%        0.097809
50%        0.178671
75%        0.286181
max        0.739130
Name: rouge_2f, dtype: float64